In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [2]:
class PerfumeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=8).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
df = pd.read_csv("data/1976_clean.csv",index_col=0)
texts = df['notes_list'].tolist()
le = LabelEncoder()
labels = le.fit_transform(df['fragrance'])
dataset = PerfumeDataset(texts, labels, tokenizer, max_len=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [5]:
def train_one_epoch():
    model.train()
    for batch in tqdm(dataloader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return loss.item()

In [6]:
def evaluate():
    model.eval()
    with torch.no_grad():
        batch = dataset[0]
        input_ids = batch['input_ids'].unsqueeze(0).to(device)
        attention_mask = batch['attention_mask'].unsqueeze(0).to(device)
        outputs = model.bert(input_ids, attention_mask=attention_mask)
        embedding = outputs.last_hidden_state[:,0,:]
        print("香水嵌入向量维度:", embedding.shape)

In [ ]:
for ep in range(1,4):
    print(f"====== Epoch {ep} ======")
    loss = train_one_epoch()
    print(f"Loss: {loss}")
    evaluate()

====== Epoch 1 ======


Training: 100%|██████████| 132/132 [00:51<00:00,  2.57it/s]


Loss: 1.3518922328948975
香水嵌入向量维度: torch.Size([1, 768])
====== Epoch 2 ======


Training: 100%|██████████| 132/132 [00:52<00:00,  2.53it/s]


Loss: 1.7422418594360352
香水嵌入向量维度: torch.Size([1, 768])
====== Epoch 3 ======


Training:  33%|███▎      | 44/132 [00:17<00:35,  2.48it/s]